<a href="https://colab.research.google.com/github/ashishsingh1331/swing/blob/main/V20_for_v200_comapnies_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Required Libraries
!pip install yfinance pandas numpy
!pip install openpyxl

# Import Libraries
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython.display import display

# List of NIFTY Next 50 companies' ticker symbols
nifty_next_50_tickers = ['NESTLEIND.NS', 'MAHAPEXLTD.NS', 'PGHH.NS', 'COLPAL.NS','LLOYDSME.NS', 'SANOFI.NS', 'LICI.NS', 'COALINDIA.NS', 'TCS.NS', 'SATINDLTD.NS', 'CASTROLIND.NS', 'IRCTC.NS', 'AIIL.NS', 'GILLETTE.NS', 'INGERRAND.NS', 'GLAXO.NS', 'ANANDRATHI.NS', 'CAMS.NS', 'MSUMI.NS', 'CGPOWER.NS', 'ABBOTINDIA.NS', 'PGHL.NS', 'GANESHHOUC.NS', 'PAGEIND.NS','MAZDOCK.NS', 'MARICO.NS', 'APARINDS.NS', 'SHARDAMOTR.NS', 'TATAELXSI.NS', 'ACE.NS', 'AKZOINDIA.NS', 'TRITURBINE.NS', 'CRISIL.NS', 'CDSL.NS', 'INFY.NS', 'OFSS.NS', 'HAL.NS', 'KPITTECH.NS', 'SHAREINDIA.NS', 'TANLA.NS', 'HDFCAMC.NS', 'ASIANPAINT.NS', 'ITC.NS', 'MGL.NS', 'NAM-INDIA.NS', 'CUMMINSIND.NS', 'ABSLAMC.NS', 'BEL.NS', 'MSTCLTD.NS','HBLPOWER.NS', 'BAYERCROP.NS', 'BAJAJ-AUTO.NS', 'LTTS.NS', 'NIITMTS.NS', 'NEULANDLAB.NS', '3MINDIA.NS', 'NBCC.NS', 'EMAMILTD.NS', 'VSTIND.NS', 'VOLTAMP.NS', 'POLYCAB.NS', 'ECLERX.NS', 'NMDC.NS', 'JWL.NS', 'AJANTPHARM.NS', 'ELECON.NS', 'LTIM.NS', 'EICHERMOT.NS', 'BCG.NS', 'ABB.NS', 'BLS.NS', 'JAMNAAUTO.NS', 'KFINTECH.NS', 'PRAJIND.NS','BSOFT.NS', 'NATCOPHARM.NS', 'HCLTECH.NS', 'HEROMOTOCO.NS', 'PIDILITIND.NS', 'GPIL.NS', 'BIKAJI.NS', 'CLEAN.NS', 'IGL.NS', 'WSTCSTPAPR.NS', 'SKFINDIA.NS', 'PERSISTENT.NS', 'FINEORG.NS', 'MANINFRA.NS', 'UNITDSPR.NS', 'SUPREMEIND.NS', 'GRSE.NS', 'COFORGE.NS', 'TATATECH.NS', 'GLS.NS', 'JYOTHYLAB.NS', 'CONCORDBIO.NS', 'RATNAMANI.NS', 'KIRLOSBROS.NS', 'BERGEPAINT.NS']

# Fetch 2 years of historical data
end_date = datetime.now()
start_date = end_date - timedelta(days=2*365)

data = {ticker: yf.download(ticker, start=start_date, end=end_date) for ticker in nifty_next_50_tickers}

# Define Functions to Identify Continuous Green Candles and Calculate Movements
def find_continuous_green_candles(data):
    green_candles = []
    for ticker, df in data.items():
        df['Green'] = df['Close'] > df['Open']
        current_streak = []
        for i in range(1, len(df)):
            if df['Green'].iloc[i]:
                current_streak.append(i)
            else:
                if len(current_streak) > 1:
                    green_candles.append((ticker, current_streak))
                current_streak = []
        if len(current_streak) > 1:
            green_candles.append((ticker, current_streak))
    return green_candles

def calculate_movement(data, green_candles):
    results = []
    for ticker, streak in green_candles:
        start_idx = streak[0]
        end_idx = streak[-1]
        lowest_price = data[ticker]['Low'].iloc[start_idx]
        highest_price = data[ticker]['High'].iloc[end_idx]
        movement = (highest_price - lowest_price) / lowest_price * 100
        if movement > 20:
            results.append({
                'Company': ticker,
                'Start Date': data[ticker].index[start_idx],
                'End Date': data[ticker].index[end_idx],
                'Lowest Price of First Green Candle': lowest_price,
                'Highest Price of Last Green Candle': highest_price,
                'Percentage Movement': movement
            })
    return pd.DataFrame(results)

green_candles = find_continuous_green_candles(data)
movement_results = calculate_movement(data, green_candles)

# Check for Current Market Price Condition and Filter Results
def filter_results(data, movement_results):
    filtered_results = []

    for i, row in movement_results.iterrows():
        ticker = row['Company']
        start_idx = data[ticker].index.get_loc(row['Start Date'])
        lowest_price = data[ticker]['Low'].iloc[start_idx]
        current_price = data[ticker]['Close'][-1]

        # Calculate 200-day moving average of closing prices
        if len(data[ticker]) > 200:
            moving_avg_200 = data[ticker]['Close'].rolling(window=200).mean().iloc[-1]
        else:
            moving_avg_200 = np.nan

        if (current_price < lowest_price) and (lowest_price < moving_avg_200):
            filtered_results.append(row)

    return pd.DataFrame(filtered_results)

filtered_results = filter_results(data, movement_results)

# Combine movement_results and filtered_results
final_results = pd.concat([movement_results, filtered_results]).drop_duplicates().reset_index(drop=True)

# Highlight rows where the lowest price of the first green candle is less than the 200-day moving average
def highlight_rows(row):
    ticker = row['Company']
    start_idx = data[ticker].index.get_loc(row['Start Date'])
    lowest_price = data[ticker]['Low'].iloc[start_idx]

    # Calculate 200-day moving average of closing prices
    if len(data[ticker]) > 200:
        moving_avg_200 = data[ticker]['Close'].rolling(window=200).mean().iloc[start_idx - 1]
    else:
        moving_avg_200 = np.nan

    if lowest_price < moving_avg_200:
        return ['background-color: orange; color: black']*len(row)
    else:
        return ['']*len(row)

# Separate highlighted rows and non-highlighted rows
highlighted_rows = final_results[final_results.apply(lambda row: highlight_rows(row)[0] != '', axis=1)]
non_highlighted_rows = final_results[final_results.apply(lambda row: highlight_rows(row)[0] == '', axis=1)]

# Combine non-highlighted rows followed by highlighted rows
final_sorted_results = pd.concat([non_highlighted_rows, highlighted_rows])

# Display the Results as Tables
print("Patterns with continuous green candles and > 20% movement:")
display(final_sorted_results.style.apply(highlight_rows, axis=1))

# Filter and display the second table
def filter_special_cases(data, movement_results):
    special_cases = []

    for i, row in movement_results.iterrows():
        ticker = row['Company']
        start_idx = data[ticker].index.get_loc(row['Start Date'])
        lowest_price = data[ticker]['Low'].iloc[start_idx]
        current_price = data[ticker]['Close'][-1]

        # Calculate 200-day moving average of closing prices
        if len(data[ticker]) > 200:
            moving_avg_200 = data[ticker]['Close'].rolling(window=200).mean().iloc[start_idx - 1]
        else:
            moving_avg_200 = np.nan

        if (current_price < lowest_price) and (lowest_price < moving_avg_200):
            special_cases.append(row)

    return pd.DataFrame(special_cases)

special_cases = filter_special_cases(data, movement_results)

print("\nCompanies with current market price lower than the lowest price of the first green candle and lowest price of the first green candle less than 200-day moving average:")
display(special_cases.style.apply(highlight_rows, axis=1))


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Patterns with continuous green candles and > 20% movement:


,Company,Start Date,End Date,Lowest Price of First Green Candle,Highest Price of Last Green Candle,Percentage Movement
0,MAHAPEXLTD.NS,2022-12-26 00:00:00,2022-12-29 00:00:00,99.000000,124.050003,25.303033
1,MAHAPEXLTD.NS,2023-06-15 00:00:00,2023-06-20 00:00:00,122.000000,151.699997,24.344260
2,MAHAPEXLTD.NS,2024-01-30 00:00:00,2024-02-01 00:00:00,138.550003,166.800003,20.389751
5,LICI.NS,2024-02-02 00:00:00,2024-02-08 00:00:00,937.000000,1145.000000,22.198506
6,SATINDLTD.NS,2023-06-30 00:00:00,2023-07-07 00:00:00,78.099998,101.500000,29.961590
7,SATINDLTD.NS,2023-08-03 00:00:00,2023-08-08 00:00:00,104.900002,126.300003,20.400382
9,CASTROLIND.NS,2023-12-21 00:00:00,2023-12-26 00:00:00,143.500000,186.000000,29.616725
10,AIIL.NS,2024-06-05 00:00:00,2024-06-07 00:00:00,795.000000,1084.449951,36.408799
11,GLAXO.NS,2023-12-21 00:00:00,2024-01-05 00:00:00,1667.849976,2157.800049,29.376148
12,ANANDRATHI.NS,2023-07-20 00:00:00,2023-08-01 00:00:00,1021.250000,1274.849976,24.832311



Companies with current market price lower than the lowest price of the first green candle and lowest price of the first green candle less than 200-day moving average:


,Company,Start Date,End Date,Lowest Price of First Green Candle,Highest Price of Last Green Candle,Percentage Movement
112,BCG.NS,2023-05-05 00:00:00,2023-05-09 00:00:00,10.250000,12.350000,20.487809
113,BCG.NS,2023-12-07 00:00:00,2023-12-08 00:00:00,17.000000,22.950001,35.000004
